In [1]:
!pwd
!ls

/content
drive  sample_data  TransFetch


In [2]:
%cd /content/

/content


In [3]:
!rm -rf TransFetch/

In [4]:
!git clone https://github.com/eshnil2000/TransFetch

Cloning into 'TransFetch'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 208 (delta 53), reused 178 (delta 37), pack-reused 0
Receiving objects: 100% (208/208), 7.79 MiB | 19.32 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [5]:
%cd /content/TransFetch/

/content/TransFetch


In [6]:
!cp /content/drive/MyDrive/data/623.xalancbmk-s5.txt.xz .
!xz --decompress 623.xalancbmk-s5.txt.xz

In [14]:
!cp /content/drive/MyDrive/data/trace-3rd-run-1M.out.tar.gz .
! tar -xvf trace-3rd-run-1M.out.tar.gz
!cp trace-3rd-run-1M.out in.txt

trace-3rd-run-1M.out


In [16]:
!cp 623.xalancbmk-s5.txt testxalanc.txt

In [31]:
!head -741262 trace-3rd-run-1M.out >test_temp.txt

In [32]:
!wc -l testxalanc.txt test_temp.txt

  741261 testxalanc.txt
  741262 test_temp.txt
 1482523 total


In [15]:
!cp trace-3rd-run-1M.out test_temp.txt

In [33]:
!sed '1d' test_temp.txt >test.txt

In [34]:
!head -5 test.txt

ts=1988336 pid=745  inst=1835086430414849 cpuid=1 rip=0x559352c81b92 va=0x7fd244d2fc30
ts=1988336 pid=745  inst=1835086448240641 cpuid=1 rip=0x559352c81b92 va=0x7fd244d2e9b0
ts=1988336 pid=745  inst=1835086452482561 cpuid=1 rip=0x559352c81b92 va=0x7fd244d2d730
ts=1988336 pid=745  inst=1835086457245697 cpuid=1 rip=0x559352c81b92 va=0x7fd244d2bb70
ts=1988336 pid=745  inst=1835086462264321 cpuid=1 rip=0x559352c81b92 va=0x7fd244d29fb0


In [35]:
import re
import csv
import random

infile='test.txt'
infile_xalanc= 'testxalanc.txt'

outfile='output.txt'
UID=[]
INST=[]
I=[]
# Read the data from the file
with open(infile, 'r') as file:
    data = file.readlines()
with open(infile_xalanc, 'r') as filexalanc:
    dataxalanc = filexalanc.readlines()
var=0
pattern = r'ts=(\d+)\s+pid=(\d+)\s+inst=(\d+)\s+cpuid=(\d+)\s+rip=([\da-fx]+)\s+va=([\da-fx]+)'
i=4
# Prepare the CSV data
csv_data = []
for line, linexalanc in zip(data, dataxalanc):
#for line in data:
    #print(line)
    #print(linexalanc)
    #match = re.match(r'ts=(\d+) mstress pid=(\d+)  inst=(\d+) cpuid=(\d+) rip=([a-f\d]+) va=([a-f\d]+)', line)
    match = re.match(pattern, line)
    #instr_ptr=linexalanc.split(sep=',')[3]
    uid=linexalanc.split(sep=',')[0]
    UID.append(uid)
    I.append(i)
    #print(linexalanc.split()[3])
    if match:
        #print('match')
        ts, pid, inst, cpuid, rip, va = match.groups()

        INST.append(inst)
        #csv_data.append([ts, pid, inst, cpuid, rip, '0x'+va])
        #csv_data.append(['0x'+va+":",'R', '0x'+va])
        #csv_data.append([va+":",'R', va])
        # Convert the hexadecimal address to an integer
        addr_int = int(va, 16)

        # Shift the address right by 12 bits
        shifted_addr_int = addr_int >> 12

        # Convert the shifted integer address back to hexadecimal format
        shifted_addr_hex = hex(shifted_addr_int)
        #shifted_addr_hex=va
        #csv_data.append([ts, pid, inst, cpuid, rip, '0x'+va])
        #Format Unique Instr Id, Cycle Count, Load Address, Instruction Pointer of the Load, LLC hit/miss
        #sample 4, 15, 28e837c88340, 406a82, 0
        modified_shifted_addr_hex = shifted_addr_hex.replace('0x', '')
        rip= rip.replace('0x', '')
        #print("shifted_addr_hex",shifted_addr_hex,"modified_shifted_addr_hex", modified_shifted_addr_hex)
        #increment = random.randint(increment_min, increment_max)
        #current_value += increment
        #increment2 = random.randint(increment_min2, increment_max2)
        #current_value2 += increment2

        #csv_data.append([str(current_value2), ' '+str(current_value), ' '+str(modified_shifted_addr_hex), ' '+str(random.choice(values)), ' '+str(0)])
        #Unique Instr ID = inst, Cycle Count = ts, Load Address = va >>12, Intruction Pointer of Load = rip, LLC hit/miss = R/W
        #csv_data.append([str(inst), ' '+str(ts), ' '+str(modified_shifted_addr_hex), ' '+str(rip), ' '+str(0)])
        csv_data.append([str(uid), ' '+str(ts), ' '+str(modified_shifted_addr_hex), ' '+str(rip), ' '+str(0)])
        #csv_data.append([str(i), ' '+str(ts), ' '+str(modified_shifted_addr_hex), ' '+str(rip), ' '+str(0)])
        i=i+1

# Write CSV data to a file
with open(outfile, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile,delimiter=",")
    #csv_writer.writerow(['ts', 'pid', 'inst', 'cpuid', 'rip', 'va'])
    csv_writer.writerows(csv_data)
    print("csv_data",csv_data)


print("CSV conversion completed.")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [36]:
!head -5 testxalanc.txt

4, 15, 28e837c88340, 406a82, 0
7, 261, 28e837c86f40, 406a8e, 0
17, 277, fdfd3a8c3bc0, 406abc, 0
29, 278, fdfd3a8c4200, 7b5749, 0
18, 610, fdfd3a8c5d80, 406abf, 0


In [37]:
!head -5 output.txt

4, 1988336, 7fd244d2f, 559352c81b92, 0
7, 1988336, 7fd244d2e, 559352c81b92, 0
17, 1988336, 7fd244d2d, 559352c81b92, 0
29, 1988336, 7fd244d2b, 559352c81b92, 0
18, 1988336, 7fd244d29, 559352c81b92, 0


In [11]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.6 MB/s eta 0:00:00


In [46]:
!rm -rf ./res

In [47]:
!./run_train.sh

TRAIN/VAL/TEST/SKIP: 2/1/2/0
output.txt
./output.txt
train and validation
preprocessing with context
preprocessing with context
-------------Data Proccessed------------
Epoch: 1 - loss: 0.5683576257 - accuracy: 0.0000000000 - test_loss: 0.4183266044 -- test accuracy: 0.0000000000
-------- Save Best Model! --------
Epoch: 2 - loss: 0.3459271115 - accuracy: 0.0022648084 - test_loss: 0.2763504684 -- test accuracy: 0.2446633826
-------- Save Best Model! --------
Epoch: 3 - loss: 0.2322846042 - accuracy: 0.1829268293 - test_loss: 0.1886542976 -- test accuracy: 0.2446633826
-------- Save Best Model! --------
Epoch: 4 - loss: 0.1628503197 - accuracy: 0.1829268293 - test_loss: 0.1353429466 -- test accuracy: 0.2446633826
-------- Save Best Model! --------
Epoch: 5 - loss: 0.1203260286 - accuracy: 0.1829268293 - test_loss: 0.1020977199 -- test accuracy: 0.2446633826
-------- Save Best Model! --------
Epoch: 6 - loss: 0.0932987653 - accuracy: 0.1829268293 - test_loss: 0.0804193139 -- test accurac